In [ ]:
from pathlib import Path, PureWindowsPath
import pandas as pd
from PIL import Image
import shutil # For copying and renaming files

In [ ]:
# Some paths
data_folder = Path("C:/Users/Bram/Documents/Somewhere/vott-csv-export")
out_folder = data_folder / 'out'
input_csv = data_folder/'vott-export.csv'

# Read the CSV file exported by the VoTT tool
df = pd.read_csv(input_csv)

In [ ]:
# Iterate over all unique images
for name, dfg in df.groupby('image'):
    # Get index from first row in the group
    index = dfg.head(1).index[0]
    img_path = data_folder/PureWindowsPath(dfg.head(1).iloc[0]['image'])
    
    # File per image with annotations in YOLO format
    file  = open(out_folder / "{:04d}.txt".format(index), 'w')
    
    # Copy file to output folder with machine friendly name
    shutil.copyfile(img_path, out_folder / "{:04d}{:s}".format(index, img_path.suffix))
    for index, row in dfg.iterrows():
        # Read bounding box (bb) coordinates
        bbw = (row["xmax"] - row["xmin"]) # Bounding box width
        bbh = (row["ymax"] - row["ymin"]) # Bounding box heigth
        bbx_centre = row["xmin"] + 0.5*bbw
        bby_centre = row["ymin"] + 0.5*bbh
        
        # Determine width and height of image
        im = Image.open(str(img_path))
        im_w, im_h = im.size

        # Relative bb coordinates
        bbw_rel = bbw/im_w
        bbh_rel = bbh/im_h
        bbx_centre_rel = bbx_centre/im_w
        bby_centre_rel = bby_centre/im_h

        # Write to file in YOLO format
        file.write("0 {:6.4f} {:6.4f} {:6.4f} {:6.4f}\n".format(bbx_centre_rel, bby_centre_rel, bbw_rel, bbh_rel))
    file.close()